In [1]:
from pycigar.envs import Env
import yaml

import numpy as np
from scipy import signal
import math
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]

class FooEnv(Env):
    @property
    def observation_space(self):
        return Box(low=-float('inf'), high=float('inf'),
                   shape=(5,), dtype=np.float64)

    @property
    def action_space(self):
        return Box(low=0.5, high=1.5, shape=(5,), dtype=np.float64)

    def step(self, rl_actions=None, randomize_rl_update=None):
        """See parent class.
        """

        for _ in range(self.sim_params['env_config']["sims_per_step"]):
            self.env_time += 1
            
            # perform action update for PV inverter device
            if len(self.k.device.group_controllers.keys()) > 0:
                control_setting = []
                devices = []
                for group_controller_name, group_controller in self.k.device.group_controllers.items():
                    action = group_controller.get_action(self)
                    if isinstance(action, tuple):
                        if isinstance(group_controller.device_id, str):
                            devices.extend([group_controller.device_id])
                            control_setting.extend((action,))
                        else:
                            devices.extend(group_controller.device_id)
                            control_setting.extend((action,)*len(group_controller.device_id))
                    elif isinstance(action, dict):
                        devices.extend(action.keys())
                        control_setting.extend(action.values())
                self.k.device.apply_control(devices, control_setting)

            # perform action update for PV inverter device
            if len(self.k.device.get_local_device_ids()) > 0:
                control_setting = []
                for device_id in self.k.device.get_local_device_ids():
                    action = self.k.device.get_controller(device_id).get_action(self)
                    control_setting.append(action)
                self.k.device.apply_control(self.k.device.get_local_device_ids(), control_setting)


            self.additional_command()

            if self.k.time <= self.k.t:
                self.k.update(reset=False)

                # check whether the simulator sucessfully solved the powerflow
                converged = self.k.simulation.check_converged()
                if not converged:
                    break

            if self.k.time >= self.k.t:
                break

        # the episode will be finished if it is not converged.
        done = not converged or (self.k.time == self.k.t)
        obs = self.get_state()
        infos = {}
        reward = self.compute_reward(rl_actions)

        return obs, reward, done, infos

    def get_state(self):
        return [0, 0, 0, 0, 0]

    def compute_reward(self, rl_actions, **kwargs):
        return 0

stream = open("./IEEE_03_Node/pycigar_config_ieee_03_cent.yaml", "r")
sim_params = yaml.safe_load(stream)

print('Simulation Complete')

Simulation Complete


In [2]:
from pycigar.utils.registry import register_devcon
from pycigar.controllers.battery_storage_controller import BatteryStorageController
from pycigar.devices.battery_storage_device import BatteryStorageDevice
from pycigar.devices.battery_storage_device_advanced import BatteryStorageDeviceAdvanced

# from pycigar.controllers.battery_peak_shaving_controller import BatteryPeakShavingController
from pycigar.controllers.battery_peak_shaving_controller_dist import BatteryPeakShavingControllerDist
from pycigar.controllers.battery_peak_shaving_controller_cent import BatteryPeakShavingControllerCent

# register_devcon('battery_storage_controller', BatteryStorageController)
# register_devcon('battery_peak_shaving_controller', BatteryPeakShavingController)
register_devcon('battery_peak_shaving_controller_dist', BatteryPeakShavingControllerDist)
register_devcon('battery_peak_shaving_controller_cent', BatteryPeakShavingControllerCent)
register_devcon('battery_storage_device_advanced', BatteryStorageDeviceAdvanced)


'battery_storage_device_advanced'

In [3]:
sim_params

{'is_disable_log': False,
 'is_disable_y': False,
 'vectorized_mode': False,
 'env_config': {'clip_actions': 'True,', 'sims_per_step': 20},
 'simulation_config': {'network_model_directory': '/home/michael/Desktop/git/ceds-cigar/pycigar/zzz/IEEE_03_Node/ieee3.dss',
  'custom_configs': {'solution_mode': 1,
   'solution_number': 1,
   'solution_step_size': 1,
   'solution_control_mode': -1,
   'solution_max_control_iterations': 1000000,
   'solution_max_iterations': 30000,
   'power_factor': 0.9}},
 'scenario_config': {'network_data_directory': '/home/michael/Desktop/git/ceds-cigar/pycigar/zzz/IEEE_03_Node/load_solar_data.csv',
  'multi_config': False,
  'start_end_time': [3600, 5401],
  'custom_configs': {'load_scaling_factor': 1.25,
   'solar_scaling_factor': 0,
   'slack_bus_voltage': 1.04,
   'load_generation_noise': False,
   'power_factor': 0.9},
  'nodes': [{'name': 's701a',
    'devices': [{'name': 'pv_1',
      'device': 'pv_device',
      'custom_device_configs': {'default_contr

In [4]:
env = FooEnv(sim_params)
env.reset()
done = False
while not done:
    _, _, done, _ = env.step()
    # if env.k.time % 60 == 0:
    #     print('Time: ' + str(env.k.time))
    # if env.k.time >= 55:
    #     done = True
    
print('Simulation Complete')

psc_1
psc_2
psc_3
bsd_1
32400000.0
bsd_2
28800000.0
bsd_3
28800000.0
bsd_1
32400000.0
bsd_2
28800000.0
bsd_3
28800000.0
Time: 51
Initialize: bsd_1
Time: 51
Controller: psc_1
Device: bsd_1
Battery SOC: 0.9
Measured active power lpf [kW]: 809.2184386867442
Measured reactive power lpf [kVAr]: 493.58042594805687
Measured apparent power lpf [kVA]: 947.869251737642

Time: 51
Initialize: bsd_2
Time: 51
Controller: psc_2
Device: bsd_2
Battery SOC: 0.8
Measured active power lpf [kW]: 809.2184386867442
Measured reactive power lpf [kVAr]: 493.58042594805687
Measured apparent power lpf [kVA]: 947.869251737642

Time: 51
Initialize: bsd_3
Time: 51
Controller: psc_3
Device: bsd_3
Battery SOC: 0.8
Measured active power lpf [kW]: 809.2184386867442
Measured reactive power lpf [kVAr]: 493.58042594805687
Measured apparent power lpf [kVA]: 947.869251737642

Time: 52
Controller: psc_1
Device: bsd_1
Battery SOC: 0.9
Measured active power lpf [kW]: 809.2249846743092
Measured reactive power lpf [kVAr]: 493.584

In [5]:
env.k.device.devices['bsd_1']['device'].current_capacity

7200000.0

In [6]:
from pycigar.utils.logging import logger

Logger = logger()

Logger.log_dict.keys()

dict_keys(['psc_1', 'psc_2', 'psc_3', 'pv_1', 'bsd_1', 'bsd_2', 'bsd_3', 's701a', 's702a', 's703a', 'network'])

In [7]:
env.k.device.devices['pv_1']['device']

In [8]:

# print(Logger.log_dict['bsd_1'].keys())
# print(Logger.log_dict['psc_1'].keys())


In [9]:
Logger.log_dict['bsd_1'].keys()
# Logger.log_dict['bsd_1']['SOC']

fig1 = plt.figure(figsize = [16, 4])
plt.plot(Logger.log_dict['psc']['p_target'],'--',label="Target Power")
plt.plot(Logger.log_dict['psc']['measured_active_power_lpf'],label="Measured Power")
# plt.plot(Logger.log_dict['psc_1']['p_target'],'--',label="Target Power")
# plt.plot(Logger.log_dict['psc_1']['measured_active_power_lpf'],label="Measured Power")
plt.title('Measured Substation Power')
plt.xlabel('Time [s]')
plt.ylabel('Power [kW]')
plt.legend()

fig1.savefig('/home/michael/Desktop/temp/ieee_03_substation_power.png')


fig1 = plt.figure(figsize = [16, 4])
plt.plot(Logger.log_dict['psc']['p_set'],label="bsd_1_psc p_set")
# plt.plot(Logger.log_dict['psc_1']['p_set'],label="psc_1 p_set")
# plt.plot(Logger.log_dict['psc_2']['p_set'],label="psc_2 p_set")
# plt.plot(Logger.log_dict['psc_3']['p_set'],label="psc_3 p_set")
plt.title('Dist Peak Shaving Controller Setpoints')
plt.xlabel('Time [s]')
plt.ylabel('PSC Setpoints [kW]')
plt.legend()

fig1.savefig('/home/michael/Desktop/temp/ieee_03_setpoint.png')


fig1 = plt.figure(figsize = [16, 4])
plt.plot(np.array(Logger.log_dict['bsd_1']['p_in'])/1e3,label="bsd_1 p_in")
plt.plot(np.array(Logger.log_dict['bsd_1']['p_out'])/1e3,label="bsd_1 p_out")
plt.plot(np.array(Logger.log_dict['bsd_2']['p_in'])/1e3,label="bsd_2 p_in")
plt.plot(np.array(Logger.log_dict['bsd_2']['p_out'])/1e3,label="bsd_2 p_out")
plt.plot(np.array(Logger.log_dict['bsd_3']['p_in'])/1e3,label="bsd_3 p_in")
plt.plot(np.array(Logger.log_dict['bsd_3']['p_out'])/1e3,label="bsd_3 p_out")
# plt.plot(Logger.log_dict['bsd_1_psc']['measured_active_power_lpf'],label="P meas lpf")
plt.title('Battery Power Output')
plt.xlabel('Time [s]')
plt.ylabel('Battery Output [kW]')
plt.legend()

fig1.savefig('/home/michael/Desktop/temp/ieee_03_battery_power.png')


fig1 = plt.figure(figsize = [16, 4])
plt.plot(Logger.log_dict['bsd_1']['SOC'],label="bsd_1 SOC")
plt.plot(Logger.log_dict['bsd_2']['SOC'],label="bsd_2 SOC")
plt.plot(Logger.log_dict['bsd_3']['SOC'],label="bsd_3 SOC")
# plt.plot(Logger.log_dict['bsd_1_psc']['measured_active_power_lpf'],label="P meas lpf")
plt.title('Battery State of Charge')
plt.xlabel('Time [s]')
plt.ylabel('SOC')
plt.legend()

fig1.savefig('/home/michael/Desktop/temp/ieee_03_battery_soc.png')

plt.show()

# fig1.savefig('/home/michael/Desktop/temp/SOC.png')

KeyError: 'psc'